# Benchmarking of integration methods - scanorama
In this notebook Scanorama integration will be tested (the actual run is performed as a slurm job)

In [1]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

/home/hers_basak/jjiang/jack/miniconda/envs/scarches/bin/python
3.9.18 (main, Sep 11 2023, 13:41:44) 
[GCC 11.2.0]
sys.version_info(major=3, minor=9, micro=18, releaselevel='final', serial=0)


In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))#embed HTML within IPython notebook and make containers 90% of screen
import numpy as np
import pandas as pd
import anndata as ad
import scanpy as sc
import seaborn as sns
import os
import matplotlib.pyplot as plt
#import scib
import scanorama
import torch
import scanpy.external as sce

%matplotlib inline
sc.settings.verbosity = 0             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=80, facecolor='white') #figure resolution and background color

# This is used
# os.path.join('data/'+object_names[i][6:10]+'_preprocessed.h5ad')
#sc.logging.print_versions()

In [3]:
#set data path
os.chdir('/home/hers_basak/jjiang/jack/outputs/deliverables/5_integration/data/scanorama')

In [4]:
files=['/home/hers_basak/jjiang/jack/outputs/deliverables/5_integration/data/complete_base.h5ad']
names=['complete']
object_names_merged=list()
numberobj = len(files)

for i in range(numberobj):
    object_names_merged.append("adata"+"_"+names[i])
    locals()[object_names_merged[i]] = sc.read(files[i]) 
    print(object_names_merged[i],' is loaded')

adata_complete  is loaded


In [5]:
atlases=['altena','agarwal','siletti','smajic','wang','welch']
for i in atlases:
    print(len(adata_complete.obs_names[adata_complete.obs['atlas']==i]))

85044
9214
287523
29495
43477
67467


Scanorama takes the atlases separately

In [6]:
sc.pp.highly_variable_genes(
    adata_complete,
    n_top_genes=2000,
    batch_key="atlas",
    subset=True)

In [7]:
batches = adata_complete.obs['atlas'].cat.categories.tolist()
alldata = {}
for batch in batches:
    alldata[batch] = adata_complete[adata_complete.obs['atlas'] == batch,]

In [8]:
adatas = list(alldata.values())

Scanorama embeds the data itself

In [9]:
#sc.tl.pca(adata_complete, svd_solver='arpack')

Batch correction should take less memory than integration

In [10]:
corrected = scanorama.correct_scanpy(adatas)

Found 2000 genes among all datasets
[[0.         0.1185153  0.48176688 0.09984806 0.13034513 0.53505535]
 [0.         0.         0.39006867 0.33097135 0.22184143 0.04070138]
 [0.         0.         0.         0.52829293 0.21836833 0.40514622]
 [0.         0.         0.         0.         0.26774029 0.21101882]
 [0.         0.         0.         0.         0.         0.241392  ]
 [0.         0.         0.         0.         0.         0.        ]]
Processing datasets (0, 5)
Processing datasets (2, 3)
Processing datasets (0, 2)
Processing datasets (2, 5)
Processing datasets (1, 2)
Processing datasets (1, 3)
Processing datasets (3, 4)
Processing datasets (4, 5)
Processing datasets (1, 4)
Processing datasets (2, 4)
Processing datasets (3, 5)
Processing datasets (0, 4)
Processing datasets (0, 1)


In [ ]:
#scanorama.integrate_scanpy(adatas, dimred = 50)

In [11]:
corrected.write('./data/adata_n30_scanorama.h5ad')

AttributeError: 'list' object has no attribute 'write'

In [ ]:
adata_complete

In [ ]:
sc.pp.neighbors(adata_complete, use_rep='X_scanorama',n_neighbors=30,)
sc.tl.leiden(adata_complete)

In [ ]:
adata_complete.write('./data/adata_n30_scanorama.h5ad')